In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore")

learning_rate = 0.01
momentum = 0.5
log_interval = 10000

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [2]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/tmp', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize((14,14)),
                                torchvision.transforms.Lambda(lambda x: torch.flatten(x)),
                             ])), shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/tmp', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize((14,14)),
                                torchvision.transforms.Lambda(lambda x: torch.flatten(x)),
                             ])), shuffle=True)

100.0%


Extracting /tmp/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/MNIST/raw



100.0%
100.0%


Extracting /tmp/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/MNIST/raw

Extracting /tmp/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/MNIST/raw



100.0%

Extracting /tmp/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/MNIST/raw



In [3]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
print(f"example_data.shape: {example_data.shape}")
print(f"example_targets.shape: {example_targets.shape}")

example_data.shape: torch.Size([1, 196])
example_targets.shape: torch.Size([1])


In [4]:
example_data.shape: torch.Size([1, 196])
example_targets.shape: torch.Size([1])

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(196, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [7]:
train_losses = []
train_counter = []

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [9]:
train(i)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.022629
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.004678
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.031699
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.007273
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.065832
Train Epoch: 0 [50000/60000 (83%)]	Loss: 0.231315


In [10]:
network.eval()

with torch.no_grad():
    pred = network(example_data)
print(f"Prediction: {pred.argmax()}")
print(f"Real Value: {example_targets.item()}")


Prediction: 3
Real Value: 3


In [11]:
torch.onnx.export(network, example_data, "mnist_pytorch.onnx")